In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [21]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [22]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [33]:
import torch

In [25]:
#read data
data = pd.read_csv('LabExamCleanO10K.csv')
dataNew = data[['Patient_ID', 'PerformedDate', 'TestResult_calc', 'Name_calc']] 

Change for different TimeDelta intervals (allowed range of time between visits)

In [24]:
lo = pd.to_timedelta('7 days')
hi = pd.to_timedelta('14 days')
print(lo, hi)

7 days 00:00:00 14 days 00:00:00


# Load in data for chosen feature

In [ ]:
#run for A1C#
dataNew = dataNew.loc[data['Name_calc'] == 'Hemoglobin A1c (HbA1c)']

In [26]:
#run for albumin/creatinine
dataNew = dataNew.loc[data['Name_calc'] == 'Urine Albumin/Creatinine Ratio (ACR)']

In [ ]:
#run for ldl
dataNew = dataNew.loc[data['Name_calc'] == 'Cholesterol in LDL (LDLc)']

In [ ]:
#run for hdl
dataNew = dataNew.loc[data['Name_calc'] == 'Cholesterol in HDL (HDLc)']

In [ ]:
#run for tri
dataNew = dataNew.loc[data['Name_calc'] == 'Triglycerides (TG)']

In [ ]:
#for sBP
data = pd.read_csv('finalNew.csv')
datanew = data.loc[data['Name_calc'] == 'sBP (mmHg)']
datanew['PerformedDate'] = pd.to_datetime(datanew['PerformedDate'], infer_datetime_format=True)
indexedDataset = datanew
filt_df = indexedDataset.loc[:, indexedDataset.columns != 'PerformedDate']
low = .25
high = .75

quant_df = filt_df.quantile([low, high])
print(quant_df)
filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)
filt_df = pd.concat([indexedDataset.loc[:,'PerformedDate'], filt_df], axis=1)
filt_df.dropna(inplace=True)
dataNew = filt_df

In [ ]:
#for dBP
data = pd.read_csv('finalNew.csv')
datanew = data.loc[data['Name_calc'] == 'dBP (mmHg)']
datanew['PerformedDate'] = pd.to_datetime(datanew['PerformedDate'], infer_datetime_format=True)
indexedDataset = datanew
filt_df = indexedDataset.loc[:, indexedDataset.columns != 'PerformedDate']
low = .25
high = .75

quant_df = filt_df.quantile([low, high])
print(quant_df)
filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)
filt_df = pd.concat([indexedDataset.loc[:,'PerformedDate'], filt_df], axis=1)
filt_df.dropna(inplace=True)
dataNew = filt_df

# Run everything after this

In [27]:
dataNew = dataNew.dropna()
import datetime
import datetime as dt
df = dataNew
df['datetime'] = dataNew['PerformedDate'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y-%m-%d'))
df = df.sort_values(by='datetime', ascending=True)
grouped = df.groupby("Patient_ID")
for name, group in grouped: 
    group['delta'] = (group['datetime']-group['datetime'].shift())
    group.drop(pd.Index(np.where((group['delta']<= lo) & (group['delta'] >=  hi))[0]), inplace = True)   

In [28]:
grouped_lists = grouped['TestResult_calc'].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['TestResult_calc'] = grouped_lists['TestResult_calc'].apply(lambda x: list(set(x)))
data = grouped_lists[['TestResult_calc']]
print(data)

                               TestResult_calc
0                                 [0.24, 1.42]
1      [4.8, 4.1, 6.6, 8.33, 10.6, 11.5, 10.0]
2             [0.9, 0.6, 1.1, 3.7, 5.51, 8.47]
3               [0.1, 0.5, 0.2, 0.3, 0.4, 0.6]
4                                        [0.8]
...                                        ...
89319                                    [0.3]
89320                                    [4.6]
89321                                    [8.4]
89322                                    [1.2]
89323                                    [6.1]

[89324 rows x 1 columns]


In [29]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['TestResult_calc'].at[index]) > 15 :
        seq = data['TestResult_calc'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]

In [30]:
 import numpy as np
>>> from sklearn.model_selection import train_test_split

In [31]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [32]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)
result = model.evaluate(X_test,y_test)
print("test loss, test acc:", result)

203/203 [==============================] - 0s 904us/step - loss: 4.5317
test loss, test acc: 4.53165340423584
